<a href="https://colab.research.google.com/github/DesmondChoy/create_adventure/blob/main/prompt_experimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install claudette -qqq

In [10]:
import os
import random
import json
from claudette.core import Chat, Client, models
from google.colab import userdata

os.environ['ANTHROPIC_LOG'] = 'debug'
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')


# Purpose: Build a MVP

This is a web-based "Choose your own adventure" RPG, but using LLMs and with an educational twist. Through the adventure, the reader is offered closed-ended player agency choices (strategic decisions, unique equipment, companion, etc).


MVP: 5 pages.

How the workflow looks like in my mind:

1. Intialization
- Pre-define data: educational trivia category, q&a pairs
- Begin by asking what type of adventure to embark on: choose_topic
- choose_topic = ['Magical Realms', 'Animal Adventures', 'Mystery and Detectives', 'Inventions and Gadgets']
- educational_trivia = random.choice(["Singapore history", "Farm animals", "Human body"])

2. Page I & II
- A function start_adventure()  takes {choose_topic}  and {educational_trivia} as arguments
- start_adventure() produces Page I and II
- Page I sets the context and character development, with no choices. The plot/narrative must be related to {educational_trivia}
- Page II concludes with a Player agency choice. Answer is saved as user_choice

3. Page III
- A function continue_adventure() takes two arguments, {story_so_far} and {user_choice}
- continue_adventure() continues to build the plot and is shaped by {user_choice}
- Page III concludes with a trivia question related to {educational_trivia}. Answer is saved as user_answer

4. Page IV
- continue_adventure() is used again but it will need to take an additional argument {user_answer} together with {story_so_far}
- continue_adventure() continues to build the plot and is shaped by whether {user_answer} was correct or incorrect. If incorrect, there should be a lesson educating the reader on the correct answer.
- Page IV concludes with a Player agency choice. Answer is saved as user_choice

5. Page V
- A function concludes adventure() takes two arguments, {story_so_far} and {user_choice}
- concludes adventure() reveals the plot and is shaped by {user_choice}. The plot is also related to {educational_trivia}.
- There will be a happy ending but there is also a hint of more adventures to come.

Some comments
- Persistence: The story state between pages is saved as string variables for now. I'll think of a better solution later down the road.

- Modularity: The focus is on shipping a MVP and refactoring will be done later. I'm open to simple, intuitive suggestions on how the functions should like like e.g. class structures

- Story generation: Claude Sonnet 3.5 will be used to generate the narrative dynamically.



To be included later:
- Difficulty level (easy/medium/hard) - this will define maths qns.
- Rendered images



# Intialization

In [19]:
import random
from claudette.core import Chat, Client, models
from IPython.display import display, HTML

# Initialize the Claude client
client = Client(models[-1])
chat = Chat(cli=client)

# Pre-define adventure topics and trivia questions (same as before)
choose_topic = ['Magical Realms', 'Animal Adventures', 'Mystery and Detectives', 'Inventions and Gadgets']
trivia_topic = random.choice(["Singapore history", "Farm animals", "Human body"])

trivia_topic_q&a = {
    "Singapore history": [
        {
            "question": "In which year did Singapore gain independence?",
            "choices": {"A": "1965", "B": "1955", "C": "1975"},
            "correct": "A"
        },
        {
            "question": "Who is considered the founding father of modern Singapore?",
            "choices": {"A": "Goh Chok Tong", "B": "Lee Kuan Yew", "C": "Tony Tan"},
            "correct": "B"
        }
    ],
    "Farm animals": [
        {
            "question": "Which farm animal provides us with wool?",
            "choices": {"A": "Cow", "B": "Chicken", "C": "Sheep"},
            "correct": "C"
        },
        {
            "question": "What do we call a baby cow?",
            "choices": {"A": "Kitten", "B": "Calf", "C": "Puppy"},
            "correct": "B"
        }
    ],
    "Human body": [
        {
            "question": "How many chambers does the human heart have?",
            "choices": {"A": "Two", "B": "Three", "C": "Four"},
            "correct": "C"
        },
        {
            "question": "What is the largest organ in the human body?",
            "choices": {"A": "Liver", "B": "Brain", "C": "Skin"},
            "correct": "C"
        }
    ]
}

In [20]:
trivia_questions

{'Singapore history': [{'question': 'In which year did Singapore gain independence?',
   'choices': {'A': '1965', 'B': '1955', 'C': '1975'},
   'correct': 'A'},
  {'question': 'Who is considered the founding father of modern Singapore?',
   'choices': {'A': 'Goh Chok Tong', 'B': 'Lee Kuan Yew', 'C': 'Tony Tan'},
   'correct': 'B'}],
 'Farm animals': [{'question': 'Which farm animal provides us with wool?',
   'choices': {'A': 'Cow', 'B': 'Chicken', 'C': 'Sheep'},
   'correct': 'C'},
  {'question': 'What do we call a baby cow?',
   'choices': {'A': 'Kitten', 'B': 'Calf', 'C': 'Puppy'},
   'correct': 'B'}],
 'Human body': [{'question': 'How many chambers does the human heart have?',
   'choices': {'A': 'Two', 'B': 'Three', 'C': 'Four'},
   'correct': 'C'},
  {'question': 'What is the largest organ in the human body?',
   'choices': {'A': 'Liver', 'B': 'Brain', 'C': 'Skin'},
   'correct': 'C'}]}

In [21]:
import pandas as pd

data = {
    'Topic': ['Singapore History'] * 5 + ['Farm Animals'] * 5 + ['Human Body'] * 5,
    'Question': [
        'What animal is the symbol of Singapore?',
        'What color is the top half of Singapore\'s flag?',
        'Who is known as the founder of modern Singapore?',
        'What year did Singapore become independent?',
        'What is the name of Singapore\'s famous airport?',
        'Which farm animal gives us milk?',
        'What do we call a baby sheep?',
        'Which farm animal says "oink oink"?',
        'What farm animal lays eggs?',
        'Which farm animal has a woolly coat?',
        'How many fingers do you have on one hand?',
        'What organ pumps blood around your body?',
        'What do you use to smell things?',
        'How many legs does a person have?',
        'What covers the outside of your body?'
    ],
    'Choice1': ['Lion', 'Red', 'Stamford Raffles', '1965', 'Changi', 'Cow', 'Lamb', 'Pig', 'Chicken', 'Sheep', '5', 'Heart', 'Nose', '2', 'Skin'],
    'Choice2': ['Tiger', 'Blue', 'Lee Kuan Yew', '1955', 'Sentosa', 'Chicken', 'Calf', 'Horse', 'Cow', 'Donkey', '4', 'Lungs', 'Ears', '3', 'Hair'],
    'Choice3': ['Elephant', 'White', 'Yusof Ishak', '1975', 'Marina Bay', 'Pig', 'Piglet', 'Goat', 'Sheep', 'Rooster', '6', 'Brain', 'Eyes', '4', 'Clothes'],
    'Correct Answer': ['Lion', 'Red', 'Stamford Raffles', '1965', 'Changi', 'Cow', 'Lamb', 'Pig', 'Chicken', 'Sheep', '5', 'Heart', 'Nose', '2', 'Skin']
}

df = pd.DataFrame(data)
print(df)


                Topic                                          Question  \
0   Singapore History           What animal is the symbol of Singapore?   
1   Singapore History   What color is the top half of Singapore's flag?   
2   Singapore History  Who is known as the founder of modern Singapore?   
3   Singapore History       What year did Singapore become independent?   
4   Singapore History   What is the name of Singapore's famous airport?   
5        Farm Animals                  Which farm animal gives us milk?   
6        Farm Animals                     What do we call a baby sheep?   
7        Farm Animals               Which farm animal says "oink oink"?   
8        Farm Animals                       What farm animal lays eggs?   
9        Farm Animals              Which farm animal has a woolly coat?   
10         Human Body         How many fingers do you have on one hand?   
11         Human Body          What organ pumps blood around your body?   
12         Human Body    